In [1]:
import os # 운영체제와 상호작용 할 수 있는 방법을 제공하는 모델
from dotenv import load_dotenv # env파일 안에있는 API키를 가져오기 위한 라이브러리

In [2]:
load_dotenv() # API키 가져오기

True

In [3]:
# langchain을 통해 OpenAI의 GPT모델 사용
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")


In [4]:
# pdf파일을 불러오기 위한 PyPDFLoader를 import
from langchain.document_loaders import PyPDFLoader

In [6]:
# pdf파일 로드
loader = PyPDFLoader("[2024 한권으로 OK 주식과 세금].pdf")

# 페이지별 문서 로드
docs = loader.load()

In [11]:
# 문서 청크로 나누기

# 방법1

# 청킹을 위한 라이브러리 imoprt
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n", # /n/n 기준으로 분할
    chunk_size=100,   # 청크 크기는 100자
    chunk_overlap=10, # 청크 간 중복된 문자는 10개
    length_function=len,  # len 함수로 텍스트의 길이 계산(문자 수 반환)
    is_separator_regex=False,  # 구분자로 정규식을 사용하지 않는다
)

splits = text_splitter.split_documents(docs)

In [8]:
print(splits[2])

page_content='주식등 과세이해를 위한 기초지식
주식등 양도소득의 이해
 1.
 2 .
 3.
 4.
 5.
 6.
 7.
 8.
 9.
10.
 
11.
12.
주식의 개념과 종류가 궁금해요
주식시장에 대해 알고 싶어요
상장이란 무엇인가요? 
주식거래 절차도 알고 싶어요 
상장주식 매매거래의 종류 및 시간은 어떻게 되나요? 
주식 매매계약은 어떻게 체결되나요?
단일가매매와 접속매매의 적용시간에 대해 알고 싶어요
기타 매매계약체결방법도 알려주실 수 있나요?
상장주식 거래 후 결제는 어떻게 하나요?
주식과 유사한 금융투자상품도 많던데...  
금융투자상품이란 무엇인가요?
금융투자상품 중 증권의 개념 및 종류에 대해 알려주세요
주식등에 부과되는 세금의 종류를 알고 싶어요
제1장
제2장
3
5
8
10
12
14
19
22
26
 
28 
29
37
37
38
38
39
42
제1절   주식등에 관련된 여러 가지 세금
제2절  주식등 양도소득 과세대상
13.
14.
15.
주식을 양도하면 세금이 나오나요?
양도소득세 과세대상인 '주식등'의 범위는 어떻게 되나요?
양도란 무엇인가요? 
목차' metadata={'source': '[2024 한권으로 OK 주식과 세금].pdf', 'page': 4}


## CharacterTextSplitter
주어진 텍스트를 기준으로 정의된 구분자를 사용하여 텍스트를 나눈다. 특정 문자를 기준으로 분할해서 문장이나 문단 단위로 텍스트를 나누는데 효과적(간단한 작업, 빠르고 직관적인 경우 유리)
>
- separator
분할된 각 청크를 구분할 때 기준이 되는 문자열
>
- chunk_size
각 청크의 최대 길이
>
- chunk_overlap
인접한 청크 사이에 중복으로 포함될 문자의 수
>
- length_function
청크의 길이를 계산하는 함수
>
- is_separator_regex
매개변수를 False로 설정하여 separator를 정규식이 아닌 일반 문자열로 처리
>
#### 각 청크의 크기가 chunk_size를 초과하지 않으며 인접한 청크 사이에는 chunk_overlap만큼의 문자가 중복되어 있게 함으로써, 텍스트의 의미적 연속성을 유지하면서 큰 데이터를 더 작은 단위로 분할할 수 있다
>
- text_splitter 
>
1. split_documents
문서(document) 파일을 load한 후 다시 작은 단위조각의 문서(document) list로 반환
>
2. split_text
문자 스트링(str)을 list로 반환
>
3. create_documents
문자열list를 문서(document)list로 반환



In [9]:
# 방법2

# 청킹을 위한 라이브러리 imoprt
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,             # 청크 크기는 100자
    chunk_overlap=10,           # 청크 간의 중복되는 문자는 10개
    length_function=len,        # len함수로 텍스트의 길이 계산
    is_separator_regex=False,   # 구분자로 정규식을 사용하지 않는다
)

splits = recursive_text_splitter.split_documents(docs)

In [10]:
print(splits[2])

page_content='쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무' metadata={'source': '[2024 한권으로 OK 주식과 세금].pdf', 'page': 2}


## RecursiveCharacterTextSplitter
여러 레벨의 구분자를 정의하여 점진적으로 텍스트를 나누는 방식(재귀적으로 분할). e.g. 문단으로 나눈 뒤 문단이 길면 문장 단위로, 문장이 여전히 길면 단어 단위로 나눔(긴 텍스트, 문맥 유지, 복잡한 텍스트 구조에 유리)
>
- chunk_size
청크 크기
>
- chunk_overlap
청크 간의 중복되는 문자 수
>
- length_function
문자열 길이를 계산하는 함수 지정
>
- is_separator_regex
구분자로 정규식을 사용할지 여부

In [12]:
# OpenAI모델을 이용한 벡터 임베딩 생성

from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [13]:
# 벡터 유사성 검색을 위한 라이브러리 FAISS

import faiss
from langchain_community.vectorstores import FAISS

# 방법2의 splits 사용, 벡터 스토어 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [14]:
# RAG체인에서 사용할 수 있도록 FAISS를 retriever로 변환 
#retriever는 vectorstore에서 데이터를 검색하는 인터페이스
# search_type 검색유형(similarity(default값), mmr(maxiaml marginal relevance)검색셜과의 관련성과 다양성을 균형있게 조정하는 방식, similarity_score_threshold 유사도 점수 임계값을 설정, 임계값 이상의 점수를 가진 문서만 반환)
# search_kwargs 검색에 대한 추가 인자. k: 최종적으로 반환할 문서 수. (lambda_mult mmr에서 반환되는 결과의 다양성, score_threshold 유사도점수 임계값 설정, fetch_k mmr에 전잘할 문서 양)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [15]:
from langchain_core.prompts import ChatPromptTemplate # 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성
from langchain_core.runnables import RunnablePassthrough # 데이터를 전달하는 역할

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([  # 전달된 메시지들을 기반으로 프롬프트를 구성
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [16]:
# LangChain 모델과 프롬프트를 연결하여 RAG 체인을 구성

# 에러 발생 시 알려줄 클래스 생성
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config(환경설절) 인수 추가
        # context의 각 문서 페이지를 줄바꿈하고 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


In [ ]:
rag_chain_debug  # | 는 다음 실행 순서를 알려주는 것

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12096ea90>, search_kwargs={'k': 1}),
  question: DebugPassThrough()
}
| DebugPassThrough()
| ContextToText()
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the question using only the following context.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Context: {context}\\n\\nQuestion: {question}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1211c3150>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1211f3210>, root_client=<openai.OpenAI object at 0x120e4ea50>, ro

In [20]:
# 챗봇 구동 확인

while True: 
	print("========================")
	query = input("질문을 입력하세요: ")
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)
	break

Debug Output: 비상장주식등에 대한 양도소득세도 대주주만 해당되나요? 과세대상에서 제외되는 경우는?
Debug Output: {'context': [Document(metadata={'source': '[2024 한권으로 OK 주식과 세금].pdf', 'page': 51}, page_content='제2장 주식등 양도소득의 이해\n1\n40\n3456789\n양도소득세 과세대상 주식등의 구체적인 범위는 다음과 같습니다.\n1) 상장주식등\n주권상장법인의 대주주가 양도하는 상장주식 및 대주주 외의 자가 증권시장에서의 거래에 \n의하지 아니하고 양도(장외거래)하는 상장주식이 과세대상에 해당합니다.\n2) 비상장주식등\n주권비상장법인 주식은 대주주 여부와 관계없이 모두 양도소득세 과세대상에 해당합니다.  \n다만, 대주주 외의 자가 K-OTC(한국금융투자협회가 운영하는 비상장법인 주식 거래 시장)를 통해 양도하는 \n중소·중견기업 주식은 과세대상에서 제외됩니다.\n3) 국외주식등\n외국법인이 발행한 주식등(국내증권시장에 상장된 주식은 제외) 과 내국법인이 발행한 경우로서 \n해외증권시장에 상장된 주식등의 경우 과세대상에 해당합니다.\n4) 신주인수권\n신주인수권이란 기업의 자금조달을 용이하게 하기 위한 목적으로 주주 및 채권자 등에게 \n부여한 신주를 인수할 수 있는 권리로서, 자금조달 방법에 따라 그 권리의 부여 형태가 \n상이합니다.\n가) 신주인수권증서\n유상증자 시 다른 사람에 비해 신주를 우선적으로 인수할 수 있는 권리를 문서화한  \n증서로, 기존주주*가 그 권리를 타인에게 양도 시 발행합니다.3)\n* 주주는 보유주식 수에 따라 신주를 배정 받을 권리가 있음(상법 §418조①)\n나) 신주인수권증권\n일정한 권리행사기간에 미리 정해진 가격으로 주식을 인수할 수 있는 권리를 문서화한 \n증서로, 신주인수권부사채(BW)* 발행 시 낮은 이자율을 대신하여 사채권과 별도로 \n채권자에게 발행합니다.4)\n* 신주인수권부사채(Bond with Warra

일반 gpt에게 질문한 경우 
```

비상장주식에 대한 양도소득세는 기본적으로 대주주와 관련이 깊지만, 대주주가 아닌 경우에도 일정 조건에 따라 과세될 수 있습니다. 양도소득세는 주식 양도 시 발생하는 이익에 대해 과세하는 세금으로, 비상장주식도 포함됩니다. 다만, 대주주 여부와 과세 여부에는 차이가 있을 수 있습니다.

1. 대주주에 해당되는 경우
대주주란 주식의 보유 비율이 일정 기준 이상인 사람을 의미합니다. 2024년 기준으로 대주주에 해당하는 기준은 다음과 같습니다:

상장주식: 1% 이상 보유
비상장주식: 2% 이상 보유
대주주가 비상장주식을 양도하는 경우에는 양도소득세가 부과됩니다. 비상장주식에 대한 양도소득세는 기본적으로 기타소득으로 분류되어, 해당 소득이 대주주의 종합소득에 포함되어 과세됩니다.

2. 대주주가 아닌 경우
비상장주식을 보유한 사람이 대주주가 아니라면, 양도소득세가 부과되는지 여부는 다음과 같은 조건에 따라 달라집니다:

양도금액이 일정 기준 이하일 경우:
비상장주식의 양도소득이 일정 금액 이하인 경우에는 양도소득세 과세대상에서 제외됩니다.
양도소득세 면세 기준은 2024년 현재로서, 비상장주식의 경우 양도금액이 3억 원 이하인 경우에는 양도소득세가 부과되지 않습니다.
예를 들어, 3억 원 이하의 금액으로 비상장주식을 양도했다면 양도소득세가 과세되지 않지만, 3억 원을 초과하는 경우에는 과세가 발생합니다.
3. 양도소득세 과세 제외되는 경우
대주주가 아니거나 양도금액이 일정 기준 이하일 때 양도소득세가 면제되는 경우가 있습니다. 대표적으로 다음과 같은 상황에서 과세가 제외될 수 있습니다:

양도금액이 3억 원 이하일 경우 (비상장주식)
상속 또는 증여로 인한 양도: 상속받은 주식이나 증여받은 주식의 경우에는 양도소득세가 부과되지 않거나, 일정한 세금 혜택이 적용될 수 있습니다.
사업용 자산의 양도: 사업에 사용된 주식 등은 일정 조건을 충족할 경우 양도소득세 과세에서 제외될 수 있습니다.
따라서, 비상장주식의 양도소득세가 과세되는지 여부는 주식 보유자 여부와 양도금액을 비롯한 다양한 요소에 따라 달라집니다
```

## 결론
위의 예시에서 알 수 있듯이 RAG를 이용한 챗봇이 정확한 답변을 해준다.
전문적인 지식 또는 최신의 정보들은 아직 챗봇에 반영이 제대로 되지 않았기 때문이다. 이를 바탕으로 전문적인 지식을 탑재한 ai를 만든다면 많은 전문직들과의 협업이 가능해 질 것 같다.